In [22]:
# Transfer Learning

In [23]:
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)
print(hub.__version__)

2.18.0
0.16.1


In [24]:
import zipfile

# Unzip the downloaded file
zip_ref = zipfile.ZipFile(r"C:\Users\HP\Downloads\10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [25]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [26]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Testing images:")
test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


In [27]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [28]:
# %pip install --upgrade tensorflow_hub

In [29]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [30]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [31]:
# import tensorflow as tf
# import tensorflow_hub as hub
# from tensorflow.keras import layers

# # Define the expected image shape (height, width)
# IMAGE_SHAPE = (224, 224)

# def create_model(model_url, num_classes=10):
#     """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
#     Args:
#       model_url (str): A TensorFlow Hub feature extraction URL.
#       num_classes (int): Number of output neurons in output layer,
#         should be equal to number of target classes, default 10.

#     Returns:
#       An uncompiled Keras Sequential model with model_url as feature
#       extractor layer and Dense output layer with num_classes outputs.
#     """
#     # Download the pretrained model and save it as a Keras layer.
#     # Note: We first create the hub layer...
#     hub_layer = hub.KerasLayer(
#         model_url,
#         trainable=False,  # freeze the underlying patterns
#         name='feature_extraction_layer',
#         input_shape=IMAGE_SHAPE + (3,)  # define the input image shape
#     )
    
#     # ... then wrap it in a Lambda layer.
#     hub_layer_wrapper = tf.keras.layers.Lambda(lambda x: hub_layer(x))
    
#     # Create our own model using the wrapped hub layer.
#     model = tf.keras.Sequential([
#         hub_layer_wrapper,  # Use the wrapped feature extraction layer
#         layers.Dense(num_classes, activation='softmax', name='output_layer')
#     ])
    
#     return model

# # Example usage:
# # model_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
# # model = create_model(model_url, num_classes=10)
# # model.build([None, 224, 224, 3])
# # model.summary()


In [32]:
# def create_model(model_url, num_classes=10):
#   """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
#   Args:
#     model_url (str): A TensorFlow Hub feature extraction URL.
#     num_classes (int): Number of output neurons in output layer,
#       should be equal to number of target classes, default 10.

#   Returns:
#     An uncompiled Keras Sequential model with model_url as feature
#     extractor layer and Dense output layer with num_classes outputs.
#   """
#   # Download the pretrained model and save it as a Keras layer

#   hub_url = model_url

# # # Create the hub layer first.
#   hub_layer = hub.KerasLayer(hub_url, trainable=False, input_shape=(224, 224, 3))

# # # Wrap the hub layer in a Lambda layer
#   hub_layer_wrapper = tf.keras.layers.Lambda(lambda x: hub_layer(x))
#   # feature_extractor_layer = hub.KerasLayer(model_url,
#   #                                          trainable=False, # freeze the underlying patterns
#   #                                          name='feature_extraction_layer',
#   #                                          input_shape=IMAGE_SHAPE+(3,)) # define the input image shape
  
#   # Create our own model
#   model = tf.keras.Sequential([
#     hub_layer_wrapper, # use the feature extraction layer as the base
#     layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer      
#   ])

#   return model

In [33]:
# # Create model
# resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

# # Compile
# resnet_model.compile(loss='categorical_crossentropy',
#                      optimizer=tf.keras.optimizers.Adam(),
#                      metrics=['accuracy'])

In [34]:
# resnet_model.summary()

In [35]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Define the expected image shape (height, width)
IMAGE_SHAPE = (224, 224)

def create_model(model_url, num_classes=10):
    """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
    
    Args:
      model_url (str): A TensorFlow Hub feature extraction URL.
      num_classes (int): Number of output neurons in output layer,
        should be equal to number of target classes, default 10.

    Returns:
      An uncompiled Keras Sequential model with model_url as feature
      extractor layer and Dense output layer with num_classes outputs.
    """
    # Create the hub layer
    hub_layer = hub.KerasLayer(
        model_url,
        trainable=False,  # freeze the underlying patterns
        name='feature_extraction_layer',
        input_shape=IMAGE_SHAPE + (3,)  # define the input image shape
    )
    
    # Wrap the hub layer in a Lambda layer
    hub_layer_wrapper = tf.keras.layers.Lambda(lambda x: hub_layer(x))
    
    # Create our own model using the wrapped hub layer.
    model = tf.keras.Sequential([
        hub_layer_wrapper,  # Wrapped feature extraction layer
        layers.Dense(num_classes, activation='softmax', name='output_layer')
    ])
    
    return model

# Example usage:
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
model = create_model(resnet_url, num_classes=10)

# Build the model by providing an input shape.
# Alternatively, you can pass a dummy input to initialize the model.
model.build(input_shape=(None, 224, 224, 3))
# Or: _ = model(tf.zeros((1, 224, 224, 3)))

model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda_5 (Lambda)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,490 (80.04 KB)

 Trainable params: 20,490 (80.04 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# import tensorflow as tf
# import tensorflow_hub as hub

# # Define your TensorFlow Hub URL. Make sure it is correct.
# hub_url = "https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-classification/1"

# # Create the hub layer first.
# hub_layer = hub.KerasLayer(hub_url, trainable=False, input_shape=(224, 224, 3))

# # Wrap the hub layer in a Lambda layer
# hub_layer_wrapper = tf.keras.layers.Lambda(lambda x: hub_layer(x))

# # Create the model using Sequential
# model = tf.keras.Sequential([
#     hub_layer_wrapper,  # Use the wrapped hub layer
#     tf.keras.layers.Dense(16, activation='relu'),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

# # Build the model to see the summary
# model.build(input_shape=(None, 224, 224, 3))
# model.summary()


In [ ]:
resnet_model = model

In [36]:
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [37]:
# Fit the model
resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  # Add TensorBoard callback to model (callbacks parameter takes a list)
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", # save experiment logs here
                                                                         experiment_name="resnet50V2")]) # name of log files

Saving TensorBoard log files to: tensorflow_hub/resnet50V2/20250303-063619


c:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.2666 - loss: 2.3666

KeyboardInterrupt: 

In [ ]:
# If you wanted to, you could really turn this into a helper function to load in with a helper.py script...
import matplotlib.pyplot as plt

# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
# If you wanted to, you could really turn this into a helper function to load in with a helper.py script...
import matplotlib.pyplot as plt

# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
# Create model
efficientnet_model = create_model(model_url=efficientnet_url, # use EfficientNetB0 TensorFlow Hub URL
                                  num_classes=train_data_10_percent.num_classes)

# Compile EfficientNet model
efficientnet_model.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

# Fit EfficientNet model 
efficientnet_history = efficientnet_model.fit(train_data_10_percent, # only use 10% of training data
                                              epochs=5, # train for 5 epochs
                                              steps_per_epoch=len(train_data_10_percent),
                                              validation_data=test_data,
                                              validation_steps=len(test_data),
                                              callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", 
                                                                                     # Track logs under different experiment name
                                                                                     experiment_name="efficientnetB0")])

In [ ]:
plot_loss_curves(efficientnet_history)

In [ ]:
efficientnet_model.summary()